In [1]:
from datetime import date

import nivapy3 as nivapy
import numpy as np
import pandas as pd

# Update reference data from Vannmiljø

This notebook updates the reference dataset used for QC in Tiltaksovervåkingen by downloading new data from Vannmiljø.

 * Make sure `all_stations_2025-11-13.xlsx` is up-to-date.

 * Update the reference period (`st_yr` and `end_yr`), if desired.

## 1. Query Vannmiljø

In [2]:
# Period to use for reference data
st_yr, end_yr = 2012, 2024

# Stations of interest
xl_path = r"../data/all_stations_2025-11-13.xlsx"
stn_df = pd.read_excel(xl_path, sheet_name="data")
stn_id_list = stn_df["vannmiljo_code"].unique().tolist()

# Filter by 'medium'
medium_list = ["Ferskvann"]

# Parameters of interest
par_list = [
    "NA",
    "MG",
    "SIO2",
    "K",
    "TOC",
    "PH",
    "SO4",
    "KOND",
    "ILAL",
    "CL",
    "N-TOT",
    "ALK",
    "N-NO3",
    "P-TOT",
    "RAL",
    "CA",
    "LAL",
    "ANC",
    "TEMP",
]

In [3]:
# Get data
data = {
    "FromDateSamplingTime": f"{st_yr}-01-01",
    "ToDateSamplingTime": f"{end_yr}-12-31",
    "FromRegDate": "1900-01-01",
    "ParameterIDFilter": par_list,
    "WaterLocationCodeFilter": stn_id_list,
}
wc_df = nivapy.da.post_data_to_vannmiljo("GetRegistrations", data=data)

# Tidy to match format expected by old code
wc_df["Betegnelse"] = np.nan
wc_df["Type"] = np.nan
wc_df["DybdeEnhet"] = np.nan
wc_df["UnntasKlassifisering"] = np.nan

names_dict = {
    "WaterLocationCode": "Vannlokalitet_kode",
    "Name": "Vannlokalitet",
    "Betegnelse": "Betegnelse",
    "Type": "Type",
    "ActivityID": "Aktivitet_id",
    "ActivityName": "Aktivitet_navn",
    "Employer": "Oppdragsgiver",
    "Contractor": "Oppdragstaker",
    "ParameterID": "Parameter_id",
    "ParameterName": "Parameter_navn",
    "CASnr": "Parameter_casnr",
    "MediumID": "Medium_id",
    "MediumName": "Medium_navn",
    "LatinskNavnID": "LatinskNavn_id",
    "VitenskapligNavn": "VitenskapligNavn",
    "SamplingMethodID": "Provetakmetode_id",
    "AnalysisMethodID": "Analysemetode_id",
    "SamplingTime": "Tid_provetak",
    "UpperDepth": "Ovre_dyp",
    "LowerDepth": "Nedre_dyp",
    "DybdeEnhet": "DybdeEnhet",
    "FilteredSample": "Filtrert_Prove",
    "UnntasKlassifisering": "UnntasKlassifisering",
    "ValueOperator": "Operator",
    "RegValue": "Verdi",
    "ListName": "Listenavn",
    "Unit": "Enhet",
    "SampleID": "Provenr",
    "DetectionLimit": "Deteksjonsgrense",
    "QuantificationLimit": "Kvantifiseringsgrense",
    "Origin": "Opprinnelse",
    "NumberOfValues": "Ant_verdier",
    "Comment": "Kommentar",
    "Archive": "Arkiv",
    "CoordX": "UTM33 Ost (X)",
    "CoordY": "UTM33 Nord (Y)",
}
wc_df = wc_df[names_dict.keys()].rename(columns=names_dict)
if medium_list:
    wc_df = wc_df.query("Medium_navn in @medium_list")
if par_list:
    wc_df = wc_df.query("Parameter_id in @par_list")
wc_df["Tid_provetak"] = pd.to_datetime(wc_df["Tid_provetak"])

# Save
date = date.today()
date = date.strftime("%Y-%m-%d")
xl_path = f"../data/vannmiljo_export_{st_yr}-{end_yr}_{date}.xlsx"
wc_df.to_excel(xl_path, sheet_name="VannmiljoEksport", index=False)

wc_df.head()

,Vannlokalitet_kode,Vannlokalitet,Betegnelse,Type,Aktivitet_id,Aktivitet_navn,Oppdragsgiver,Oppdragstaker,Parameter_id,Parameter_navn,...,Enhet,Provenr,Deteksjonsgrense,Kvantifiseringsgrense,Opprinnelse,Ant_verdier,Kommentar,Arkiv,UTM33 Ost (X),UTM33 Nord (Y)
0,027-28435,Bjerkreimselva ved Gjedrem (VK4),NaN,NaN,KALK,Tiltaksovervåking i kalkede laksevassdrag,Miljødirektoratet,NIVA,ALK,Total alkalitet,...,mmol/l,,0.01,0.033,None,1,,N,-16580.2193,6.530981e+06
1,027-28435,Bjerkreimselva ved Gjedrem (VK4),NaN,NaN,KALK,Tiltaksovervåking i kalkede laksevassdrag,Miljødirektoratet,NIVA,ALK,Total alkalitet,...,mmol/l,,0.01,0.033,None,1,,N,-16580.2193,6.530981e+06
2,027-28435,Bjerkreimselva ved Gjedrem (VK4),NaN,NaN,KALK,Tiltaksovervåking i kalkede laksevassdrag,Miljødirektoratet,NIVA,ALK,Total alkalitet,...,mmol/l,,0.01,0.033,None,1,,N,-16580.2193,6.530981e+06
3,027-28435,Bjerkreimselva ved Gjedrem (VK4),NaN,NaN,KALK,Tiltaksovervåking i kalkede laksevassdrag,Miljødirektoratet,NIVA,ALK,Total alkalitet,...,mmol/l,,0.01,0.033,None,1,,N,-16580.2193,6.530981e+06
4,027-28435,Bjerkreimselva ved Gjedrem (VK4),NaN,NaN,KALK,Tiltaksovervåking i kalkede laksevassdrag,Miljødirektoratet,NIVA,ALK,Total alkalitet,...,mmol/l,,0.01,0.033,None,1,,N,-16580.2193,6.530981e+06
